In [3]:
import os
# from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
# from sklearn.metrics import f1_score
from torchvision import datasets, transforms
# from tqdm import tqdm

class ConvBlock(nn.Module):
    def __init__(self, in_channel, mid_channel, out_channel):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channel, mid_channel, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(mid_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(mid_channel, out_channel, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channel)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        
        x = self.pool(x)
        
        return x
class LinearBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.fc = nn.Linear(in_channel, out_channel)
        self.batch_norm = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        
        return x
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convblock1 = ConvBlock(1, 32, 64)
        self.convblock2 = ConvBlock(64, 128, 128)
        self.convblock3 = ConvBlock(128, 256, 256)
        self.convblock4 = ConvBlock(256, 512, 512)
        
        self.flatten = nn.Flatten(start_dim=1)
        
        self.linearblock1 = LinearBlock(512 * 15 * 15, 1024)
        self.linearblock2 = LinearBlock(1024, 512)
        self.linearblock3 = LinearBlock(512, 16)
        
        self.linearblock4 = LinearBlock(16 + 2, 4)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, img, mean, std):
        x = self.convblock1(img)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        
        x = self.flatten(x)
        
        x = self.linearblock1(x)
        x = self.linearblock2(x)
        x = self.linearblock3(x)
        
        x = torch.concat([x, mean.unsqueeze(1), std.unsqueeze(1)], dim=-1)
        x = self.linearblock4(x)
        x = self.softmax(x)
        
        return x

    
model = CNN()

# Load the state dictionary
model.load_state_dict(torch.load('/kaggle/input/model/pytorch/default/1/CNN_2024_10_13-15_18_42.pt'))

# Set the model to evaluation mode
model.eval()

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the quantized model
torch.save(quantized_model, 'model_quantized.pt')

/tmp/ipykernel_30/3840961475.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/model/pytorch/default/1/CNN_2024_10_13-15_1

In [7]:
# Define your mean and std (these are just example values; adjust as needed)
mean = torch.tensor([0.485])  # Mean for single channel
std = torch.tensor([0.229])    # Std for single channel

# Create example input with 1 channel
example_input = torch.randn(1, 1, 240, 240)  # Shape: (batch_size, channels, height, width)

# Trace the model, providing mean and std as additional inputs
traced_model = torch.jit.trace(quantized_model, (example_input, mean, std))

# Save the traced model
traced_model.save('model_final.pt')


In [32]:
import torch.quantization
import os
# from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
# from sklearn.metrics import f1_score
from torchvision import datasets, transforms
# from tqdm import tqdm

class ConvBlock(nn.Module):
    def __init__(self, in_channel, mid_channel, out_channel):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channel, mid_channel, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(mid_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(mid_channel, out_channel, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channel)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        
        x = self.pool(x)
        
        return x
class LinearBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.fc = nn.Linear(in_channel, out_channel)
        self.batch_norm = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        
        return x
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convblock1 = ConvBlock(1, 32, 64)
        self.convblock2 = ConvBlock(64, 128, 128)
        self.convblock3 = ConvBlock(128, 256, 256)
        self.convblock4 = ConvBlock(256, 512, 512)
        
        self.flatten = nn.Flatten(start_dim=1)
        
        self.linearblock1 = LinearBlock(512 * 15 * 15, 1024)
        self.linearblock2 = LinearBlock(1024, 512)
        self.linearblock3 = LinearBlock(512, 16)
        
        self.linearblock4 = LinearBlock(16 + 2, 4)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, img, mean, std):
        x = self.convblock1(img)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        
        x = self.flatten(x)
        
        x = self.linearblock1(x)
        x = self.linearblock2(x)
        x = self.linearblock3(x)
        
        x = torch.concat([x, mean.unsqueeze(1), std.unsqueeze(1)], dim=-1)
        x = self.linearblock4(x)
        x = self.softmax(x)
        
        return x

    
model = CNN()
# Set the model to evaluation mode and prepare it for quantization
model.eval()

# Fuse modules (if applicable)
# model.fuse_model()  # Uncomment if you have fusable layers like Conv2d and ReLU

# Specify the quantization configuration
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare the model for quantization
torch.quantization.prepare(model, inplace=True)

# Calibrate the model with representative data
# Here you would typically run some data through the model to collect statistics
# for example:
# for data in calibration_data_loader:
#     model(data)

# Convert to a quantized model
quantized_model = torch.quantization.convert(model, inplace=True)

# Save the quantized model
torch.save(quantized_model,'quantized_model_2.pt')


In [34]:
import torch.nn.utils.prune as prune

# Apply pruning to your model layers
for layer in quantized_model.modules():
    if isinstance(layer, nn.Conv2d):
        prune.ln_structured(layer, name='weight', amount=0.2, n=2, dim=0)  # 20% pruning

torch.save(quantized_model,'quantized_model_3.pt')

In [1]:
%%writefile preprocess_and_predict.py 
from PIL import Image

def get_transforms():
    transform = transforms.Compose([
        transforms.Resize((248, 248)),
        transforms.ToTensor(),
    ])

    return transform

def preprocess_and_predict(model, image_path, device):
    """
    Preprocess a single image and make a prediction using the trained model.

    Args:
        model: The trained CNN model.
        image_path (str): Path to the image file.
        device: The device to run the model on (CPU or GPU).

    Returns:
        predicted_class (int): The predicted class index.
        mean (float): The mean value of the image tensor.
        std (float): The standard deviation of the image tensor.
    """
    # Load the image
    img = Image.open(image_path).convert('L')  # Convert image to grayscale

    # Transform the image
    transform = get_transforms()
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Move tensor to the specified device
    img_tensor = img_tensor.to(device)

    # Compute mean and std for the image
    mean = torch.mean(img_tensor)
    std = torch.std(img_tensor)

    # Make a prediction
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(img_tensor, mean.unsqueeze(0).to(device), std.unsqueeze(0).to(device))  # Include mean and std in the prediction
        predicted_class = output.argmax(dim=1).item()  # Get the predicted class index

    return predicted_class, mean.item(), std.item()

# Example usage
image_path = '/kaggle/input/imagesoasis/Data/Very mild Dementia/OAS1_0003_MR1_mpr-1_102.jpg'
predicted_class_index, mean, std = preprocess_and_predict(traced_model, image_path, device)

print(f"Predicted Class Index: {predicted_class_index}, Mean: {mean}, Std: {std}")
if predicted_class_index == 0 :
    print ('Mild Dementia')
    
elif predicted_class_index ==1 :
    print('Moderate Dementia')
    
elif predicted_class_index == 2 :
    print ('Non Demented')
    
elif predicted_class_index ==3 :
    print('Very mild Dementia')
    
else :
    print('not supported')

Writing preprocess_and_predict.py


In [ ]:
    def __init__(self, train_batch_size=64, test_batch_size=64, learning_rate=0.001, num_epochs=10, val_split=0.15, test_split=0.15, model_path='saved_model', dataset_path='/kaggle/input/imagesoasis/Data'):
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.val_split = val_split
        self.test_split = test_split
        self.model_path = model_path
        self.dataset_path = dataset_path
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
            mean = torch.mean(x)
            std = torch.std(x)
        return x, mean, std, y
    
    def __len__(self):
        return len(self.subset)
def get_transforms():
    transform = transforms.Compose([
        transforms.Resize((248, 248)),
        transforms.ToTensor(),
    ])

    return transform
def get_sample_weights(dataset, train_dataset):
    
    # Code taken from:
    #     https://www.maskaravivek.com/post/pytorch-weighted-random-sampler/
    y_train_indices = train_dataset.indices
    y_train = [dataset.targets[i] for i in y_train_indices]
    
    class_sample_counts = np.array([len(np.where(y_train == t)[0]) for t in np.unique(y_train)])
    
    weights = 1. / class_sample_counts
    sample_weights = np.array([weights[t] for t in y_train])
    sample_weights = torch.from_numpy(sample_weights)
    
    return sample_weights
def get_data_loaders(hparams):
    # Loading the dataset
    dataset = datasets.ImageFolder(hparams.dataset_path,
                                   transform=transforms.Compose([transforms.Grayscale()]))
    
    # Splitting dataset into train, validation and test partitions.
    proportions = [(1 - hparams.val_split - hparams.test_split), hparams.val_split, hparams.test_split]
    lengths = [int(p * len(dataset)) for p in proportions]
    lengths[-1] = len(dataset) - sum(lengths[:-1])
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, lengths)
    
    print(f'train size: {lengths[0]}, val size: {lengths[1]}, test size: {lengths[2]}')
    
    data_transforms = {
        'train': get_transforms(),
        'test': get_transforms()
    }
    
    # Using WeightedRandomSampler to overcome unbalance problem
    sample_weights = get_sample_weights(dataset, train_dataset)
    train_sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights.type('torch.DoubleTensor'), len(sample_weights))
    
    train_dataset = CustomDataset(train_dataset, transform=data_transforms['train'])
    val_dataset = CustomDataset(val_dataset, transform=data_transforms['test'])
    test_dataset = CustomDataset(test_dataset, transform=data_transforms['test'])
    
    # Creating loaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams.train_batch_size, sampler=train_sampler, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams.train_batch_size, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=hparams.test_batch_size)

    return train_loader, val_loader, test_loader

def predict(model, data_loader, criterion, device, eval=False):
    model.eval()
    pred_loss = 0
    pred_correct = 0
    total_size = 0

    predictions = torch.IntTensor()
    ground_truths = torch.IntTensor()

    predictions, ground_truths = predictions.to(device), ground_truths.to(device)

    with torch.no_grad():
        for batch_idx, (img, mean, std, target) in enumerate(data_loader):
            img, mean, std, target = img.to(device), mean.to(device), std.to(device), target.to(device)
            output = model(img, mean, std)
            loss = criterion(output, target)
            pred_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            pred_correct += pred.eq(target.view_as(pred)).sum().item()

            predictions = torch.cat((predictions, pred), dim=0)
            ground_truths = torch.cat((ground_truths, target), dim=0)
            
            total_size += len(img)
    
    pred_loss /= total_size
    pred_accuracy = 100. * pred_correct / total_size

    if eval:
        return pred_loss, pred_accuracy, predictions.cpu().numpy(), ground_truths.cpu().numpy()
    else:
        return predictions.cpu().numpy(), ground_truths.cpu().numpy()
